In [12]:
import os
import random
import pandas as pd
import numpy as np
import pytorch_lightning as pl
import torch
import argparse
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
)
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
from transformers.optimization import get_cosine_with_hard_restarts_schedule_with_warmup
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


### 1. Dataset & Tokenization

In [3]:
class news_dataset(torch.utils.data.Dataset):
    def __init__(self, news_dataset, labels):
        self.dataset = news_dataset
        self.labels = labels

    def __getitem__(self,idx):
        item = {
            key: val[idx].clone().detach() for key, val in self.dataset.items()
        }
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [4]:
def load_data(dataset_dir):
    dataset = pd.read_csv(dataset_dir)[:500]
    print("dataframe 의 형태")
    print(dataset.head())
    return dataset

def construct_tokenized_dataset(dataset,tokenizer, max_length):
    concat_entity = []
    for title, body in zip(dataset["newsTitle"],dataset["newsContent"]):
        total = str(title) + "[SEP]" + str(body)
        concat_entity.append(total)
    print("tokenizer 에 들어가는 데이터 형태")
    print(concat_entity[:5])
    tokenized_senetences = tokenizer(
        concat_entity,
        return_tensors = "pt",
        padding = True,
        truncation = True,
        max_length = max_length,
        add_special_tokens = True,
        return_token_type_ids=False, # BERT 이후 모델(RoBERTa 등) 사용할때 On
    )
    print("tokenizing 된 데이터 형태")
    print(tokenized_senetences[:5])
    return tokenized_senetences

In [5]:
def prepare_dataset(dataset_dir, tokenizer,max_len):
    # load_data
    train_dataset = load_data(os.path.join(dataset_dir, "train.csv"))
    test_dataset = load_data(os.path.join(dataset_dir, "test.csv"))

    # split train / validation = 7.5 : 2.5
    train_dataset, val_dataset = train_test_split(train_dataset,test_size=0.25,random_state=42,stratify=train_dataset['label'])

    # split label
    train_label = train_dataset['label'].values
    val_label = val_dataset['label'].values
    test_label = test_dataset['label'].values

    # tokenizing dataset
    tokenized_train = construct_tokenized_dataset(train_dataset, tokenizer, max_len)
    tokenized_val = construct_tokenized_dataset(val_dataset, tokenizer, max_len)
    tokenized_test = construct_tokenized_dataset(test_dataset, tokenizer, max_len)
    print("--- tokenizing Done ---")

    # make dataset for pytorch.
    news_train_dataset = news_dataset(tokenized_train, train_label)
    news_val_dataset = news_dataset(tokenized_val, val_label)
    news_test_dataset = news_dataset(tokenized_test, test_label)
    print("--- dataset class Done ---")

    return news_train_dataset , news_val_dataset, news_test_dataset , test_dataset

### Model & Trainer

In [6]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)

    # calculate f1 score using sklearn's function
    f1 = f1_score(labels, preds, average='micro')

    return {"accuracy": acc, "f1": f1}

In [7]:
def load_tokenizer_and_model_for_train():
    # load model and tokenizer
    MODEL_NAME = args.model_name
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # setting model hyperparameter
    model_config = AutoConfig.from_pretrained(MODEL_NAME)
    model_config.num_labels = 2
    print(model_config)

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME, config=model_config
    )
    print("--- Modeling Done ---")
    return tokenizer , model

In [8]:
def load_trainer_for_train(model,news_train_dataset,news_val_dataset):
    training_args = TrainingArguments(
        output_dir=args.save_path + "results",  # output directory
        save_total_limit=args.save_limit,  # number of total save model.
        save_steps=args.save_step,  # model saving step.
        num_train_epochs=args.epochs,  # total number of training epochs
        learning_rate=args.lr,  # learning_rate
        per_device_train_batch_size=args.batch_size,  # batch size per device during training
        per_device_eval_batch_size=2,  # batch size for evaluation
        warmup_steps=args.warmup_steps,  # number of warmup steps for learning rate scheduler
        weight_decay=args.weight_decay,  # strength of weight decay
        logging_dir=args.save_path + "logs",  # directory for storing logs
        logging_steps=args.logging_step,  # log saving step.
        eval_strategy="steps",  # evaluation strategy to adopt during training
            # `no`: No evaluation during training.
            # `steps`: Evaluate every `eval_steps`.
            # `epoch`: Evaluate every end of epoch.
        eval_steps=args.eval_step,  # evaluation step.
        load_best_model_at_end=True,
    )

    ## Add callback & optimizer & scheduler
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=3, early_stopping_threshold=0.001
    )

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=args.lr,
        betas=(0.9, 0.999),
        eps=1e-08,
        weight_decay=args.weight_decay,
        amsgrad=False,
    )
    print("--- Set training arguments Done ---")

    trainer = Trainer(
        model=model,  # the instantiated 🤗 Transformers model to be trained
        args=training_args,  # training arguments, defined above
        train_dataset=news_train_dataset,  # training dataset
        eval_dataset=news_val_dataset,  # evaluation dataset
        compute_metrics=compute_metrics,  # define metrics function
        callbacks=[MyCallback],
        optimizers=(
            optimizer,
            get_cosine_with_hard_restarts_schedule_with_warmup(
                    optimizer,
                    num_warmup_steps=args.warmup_steps,
                    num_training_steps=len(news_train_dataset) * args.epochs,
            ),
        ),
    )
    print("--- Set Trainer Done ---")

    return trainer


In [9]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [10]:
def train():
    # fix a seed
    pl.seed_everything(seed=42, workers=False)

    # set device
    if torch.backends.mps.is_available():
        device = torch.device("mps")
        print("MPS device found. Using:", device)
    else:
        device = torch.device("cpu")
        print("MPS device not found. Using CPU:", device)

    # set model and tokenizer
    tokenizer , model = load_tokenizer_and_model_for_train()
    model.to(device)

    # set data
    news_train_dataset, news_val_dataset, news_test_dataset, test_dataset = prepare_dataset(args.dataset_dir,tokenizer,args.max_len)

    # set trainer
    trainer = load_trainer_for_train(model,news_train_dataset,news_val_dataset)

    # train model
    print("--- Start train ---")
    trainer.train()
    print("--- Finish train ---")
    model.save_pretrained("./best_model")


In [11]:
class args ():
  """학습(train)과 추론(infer)에 사용되는 arguments 관리하는 class"""
  dataset_dir = "./"
  model_type = "roberta" # 다른 모델 사용 가능 e.g) "bert" , "electra" ···
  model_name = "klue/roberta-large" # 다른 모델 사용 가능 e.g) "klue/bert-base" , "monologg/koelectra-base-finetuned-nsmc" ···
  save_path = "./"
  save_step = 200
  logging_step = 200
  eval_step = 100
  save_limit = 5
  seed = 42
  epochs = 1 # 10
  batch_size = 8 # 메모리 상황에 맞게 조절 e.g) 16 or 32
  max_len = 256
  lr = 3e-5
  weight_decay = 0.01
  warmup_steps = 300
  scheduler = "linear"
  model_dir = "./best_model" #추론 시, 저장된 모델 불러오는 경로 설정

train()

Seed set to 42


MPS device found. Using: mps
RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.53.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 32000
}



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--- Modeling Done ---
dataframe 의 형태
   Unnamed: 0 newsCategory newsSubcategory                        newsTitle  \
0       12529           경제           금융/증권  김종민 대전산단협회장 “산단 발전과 인식변화 위해 노력”   
1        4815           경제              유통     힐튼 가든 인, 내달 1일 루프톱 야외 수영장 개장   
2       11556           경제           유통/쇼핑                    이마트에서 겨울패션 충전   
3       11798           경제           유통/쇼핑      새 통큰상품 기대하세요…롯데마트 창립17주년 행사   
4       15381           경제            대덕특구              추석 열차 승차권 10~11일 예매   

                                         newsContent  label  
0  “대전산업단지 발전과 인식 변화를 위해 최선의 노력을 다하겠다.” 제14대 대전산업...      1  
1  힐튼 가든 인 서울 강남은 다음달 1일부터 루프톱 야외 수영장을 오픈, 온수풀을 가...      0  
2  이마트가 지속되는 한파에 끝이 보이지 않는 겨울을 이기기 위한 비법으로 겨울 패션 ...      1  
3  롯데마트는 창립 17주년을 맞아 다양한 '통큰 상품'을 새롭게 출시한다고 26일 밝...      1  
4  올해 추석 연휴(9월 9~14일) 열차 승차권(좌석지정 승차권) 예매가 오는 10일...      1  
dataframe 의 형태
   Unnamed: 0 newsCategory newsSubcategory  \
0       21851           경제              산업   
1        

/Users/mungughyeon/miniforge3/envs/Upstage/lib/python3.9/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss


--- Finish train ---


### Inference & Evaluation

In [13]:
def load_model_for_inference():
    # load tokenizer
    Tokenizer_NAME = args.model_name
    tokenizer = AutoTokenizer.from_pretrained(Tokenizer_NAME)

    ## load my model
    model = AutoModelForSequenceClassification.from_pretrained(args.model_dir)

    return tokenizer, model

In [14]:
def inference(model, tokenized_sent, device):
    dataloader = DataLoader(tokenized_sent, batch_size=args.batch_size, shuffle=False)
    model.eval()
    output_pred = []
    for i, data in enumerate(tqdm(dataloader)):
        with torch.no_grad():
            outputs = model(
                input_ids=data["input_ids"].to(device),
                attention_mask=data["attention_mask"].to(device),
            )
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        result = np.argmax(logits, axis=-1)

        output_pred.append(result)
    return (np.concatenate(output_pred).tolist(),)

In [15]:
def infer_and_eval():
    # set device
    if torch.backends.mps.is_available():
        device = torch.device("mps")
        print("MPS device found. Using:", device)
    else:
        device = torch.device("cpu")
        print("MPS device not found. Using CPU:", device)

    # set model & tokenizer
    tokenizer, model = load_model_for_inference()
    model.to(device)

    # set data
    news_train_dataset, news_val_dataset, news_test_dataset, test_dataset = prepare_dataset(args.dataset_dir,tokenizer,args.max_len)

    # predict answer
    pred_answer = inference(model, news_test_dataset, device)  # model에서 class 추론
    print("--- Prediction done ---")

    # evaluate between label and prediction
    labels = test_dataset['label'].values
    pred = pred_answer[0]

    acc = accuracy_score(labels, pred)
    f1 = f1_score(labels, pred, average='macro')
    print(f" ----- accuracy:{acc * 100:.1f}% -----")
    print(f"----- f1_score(macro): {f1 * 100:.1f}% ------")

    # make csv file with predicted answer
    output = pd.DataFrame(
        {
            "title": test_dataset["newsTitle"],
            "cleanBody": test_dataset["newsContent"],
            "clickbaitClass": pred,
        }
    )

    # 최종적으로 완성된 예측한 라벨 csv 파일 형태로 저장.
    result_path = "./prediction/"
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    output.to_csv(
        os.path.join(result_path,"result.csv"), index=False
    )
    print("--- Save result ---")
    return output

In [16]:
output_df = infer_and_eval()
output_df.head(10)

MPS device found. Using: mps
dataframe 의 형태
   Unnamed: 0 newsCategory newsSubcategory                        newsTitle  \
0       12529           경제           금융/증권  김종민 대전산단협회장 “산단 발전과 인식변화 위해 노력”   
1        4815           경제              유통     힐튼 가든 인, 내달 1일 루프톱 야외 수영장 개장   
2       11556           경제           유통/쇼핑                    이마트에서 겨울패션 충전   
3       11798           경제           유통/쇼핑      새 통큰상품 기대하세요…롯데마트 창립17주년 행사   
4       15381           경제            대덕특구              추석 열차 승차권 10~11일 예매   

                                         newsContent  label  
0  “대전산업단지 발전과 인식 변화를 위해 최선의 노력을 다하겠다.” 제14대 대전산업...      1  
1  힐튼 가든 인 서울 강남은 다음달 1일부터 루프톱 야외 수영장을 오픈, 온수풀을 가...      0  
2  이마트가 지속되는 한파에 끝이 보이지 않는 겨울을 이기기 위한 비법으로 겨울 패션 ...      1  
3  롯데마트는 창립 17주년을 맞아 다양한 '통큰 상품'을 새롭게 출시한다고 26일 밝...      1  
4  올해 추석 연휴(9월 9~14일) 열차 승차권(좌석지정 승차권) 예매가 오는 10일...      1  
dataframe 의 형태
   Unnamed: 0 newsCategory newsSubcategory  \
0       21851           경제              산업   
1 

  0%|          | 0/63 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 63/63 [00:42<00:00,  1.48it/s]

--- Prediction done ---
 ----- accuracy:60.6% -----
----- f1_score(macro): 37.7% ------
--- Save result ---


,title,cleanBody,clickbaitClass
0,"본그룹 \""일상 속 걷기로 비대면 기부 캠페인 참여해요\""",국내 대표 한식 프랜차이즈 기업 본아이에프가 속한 '본그룹'이 오늘부터 내달 16일...,1
1,"\""그림 완성하고 미니 콘서트도\"" 아이파크몰, 체험형 콘텐츠 강화",HDC아이파크몰은 고객이 직접 참여하고 경험할 수 있는 체험형 콘텐츠를 통해 오프라...,1
2,"LH 미분양아파트 소진 '진땀', 공급실적 올리기위해 알선수수료까지 지원키로",연말을 앞두고 한국토지주택공사(LH) 대전충남지역본부가 아파트 공급실적을 끌어올리기...,1
3,"신한銀, 비대면 '땡겨요 사업자 대출' 출시…최대 1000만원 가능",신한은행은 시중은행 최초로 땡겨요 입점 개인사업자를 위한 신용대출 상품인 '땡겨요 ...,1
4,"[신년사]김현수 농식품부 장관 \""농업, 데이터 기반 첨단산업 변모\""",김현수 농림축산식품부 장관은 농업을 데이터 기반의 첨단산업으로 변모시키고 젊은 인재...,1
5,충청권 지방은행 설립 속도낸다,충청권 지방은행 설립을 위한 대전시와 지역경제계의 발걸음이 빨라질 것으로 보인다.\...,1
6,지역 하반기 고용시장 '쾌청',올해 하반기 대전·충청지역의 고용시장이 활기를 띨 전망이다.\n12일 대전고용노동청...,1
7,"삼성전자, 산학협력으로 모바일 기술 개발·인재 육성 가속",삼성전자가 최첨단 모바일 기술 개발과 인재 육성에 속도를 내기 위해 국내 대학들과의...,1
8,"이영 중기부 장관 후보 \""정치하기 위해 스펙 쌓기·기업 이용 안했다\""","이영 중소벤처기업부 장관 후보자는 11일 \""정치를 하기 위해 스펙 쌓기를 하거나 ...",1
9,지방은행 설립 세종시도 힘 보탠다,대전시가 추진중인 지방은행 부활 문제가 전국적 이슈로 부상하는 가운데 세종시까지 이...,1
